In [153]:
!pip install optuna -q

In [178]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer

import optuna

import sklearn
from sklearn import linear_model as lm

In [179]:

X_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0).values
X_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0)
y['Bound'] = y.Bound.apply(lambda x: -1 if x == 0 else 1)
y=y.values
# y.head()

In [180]:
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def compare(beta1, beta2):
    print('''
Difference between the two:
{}
        '''.format(np.sum((beta1-beta2)**2))
    )

In [181]:
X_train.shape,X_test.shape

((2000, 1), (1000, 1))

In [182]:
n = 6


X_te = []
X = []


cv = CountVectorizer()
for i in X_train:
    sentence = ' '.join(getKmers(i[0], size=n))
    X.append(sentence)
    
for i in X_test:
    sentence = ' '.join(getKmers(i[0], size=n))
    X_te.append(sentence)

    
X_later = X+X_te
X = cv.fit_transform(X+X_te).toarray()
# breakS
    

In [183]:
X[:15,:15]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]])

In [184]:
print('x_train: {} y_train {}'.format(X[:2000,:].shape,y.shape))

# print('x_train: {} y_train {}'.format(X_train.shape,y.shape))
# print('x_test: {}'.format(X_te.shape))

x_train: (2000, 4096) y_train (2000, 1)


In [119]:
X_tr= scale(X[:2000,:])
X_te = scale(X[2000:,:])

X_train, X_val, y_train, y_val = train_test_split(
    X_tr, y, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape,X_test.shape)

(1340, 4096) (660, 4096) (1340, 1) (660, 1) (1000, 1)


### Model Test

In [120]:
# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam * n
    b = X.T.dot(y)
    
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

In [121]:
lam = 10

# Our solver
beta1 = solveRR(y_train, X_train, lam)

# Python solver
alpha = lam * X_train.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X_train,y_train).coef_

# Check
compare(beta1, beta2)


Difference between the two:
140.08201928619016
        


# Ridge Regression (RR)

In [122]:
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam * n
    b = X.T.dot(y)
    
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

In [156]:
def objective(trial):
    lam = trial.suggest_loguniform('lam', 0.01, 5)

    # Our solver
    beta1 = solveRR(y_train, X_train, lam)

    # Python solver
    alpha = lam *  X_train.shape[0]
    model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
    beta2 = model.fit(X_train,y_train).coef_
    
    valid_acc = sum(np.sign(np.dot(X_val,beta1))==y_val)/len(y_val)
    train_acc = sum(np.sign(np.dot(X_train,beta1))==y_train)/len(y_train)
    
    return valid_acc

In [157]:

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=200,show_progress_bar=False)

[I 2020-05-26 10:37:06,145] Finished trial#0 with value: 0.6242424242424243 with parameters: {'lam': 0.05951581930570455}. Best is trial#0 with value: 0.6242424242424243.
[I 2020-05-26 10:37:07,770] Finished trial#1 with value: 0.6075757575757575 with parameters: {'lam': 0.010387236057764784}. Best is trial#0 with value: 0.6242424242424243.
[I 2020-05-26 10:37:09,427] Finished trial#2 with value: 0.6257575757575757 with parameters: {'lam': 0.21399836957829915}. Best is trial#2 with value: 0.6257575757575757.
[I 2020-05-26 10:37:11,052] Finished trial#3 with value: 0.6287878787878788 with parameters: {'lam': 0.8666359519782804}. Best is trial#3 with value: 0.6287878787878788.
[I 2020-05-26 10:37:12,707] Finished trial#4 with value: 0.6166666666666667 with parameters: {'lam': 0.04278848210905315}. Best is trial#3 with value: 0.6287878787878788.
[I 2020-05-26 10:37:14,327] Finished trial#5 with value: 0.6257575757575757 with parameters: {'lam': 0.17405934768917147}. Best is trial#3 with v

KeyboardInterrupt: 

In [158]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)

df.sort_values(by=['value'])

,number,value,duration,params_lam
1,1,0.607576,00:00:01.623944,0.010387
79,79,0.609091,00:00:01.605914,0.011970
9,9,0.610606,00:00:01.632601,0.013856
30,30,0.613636,00:00:01.603322,0.025540
20,20,0.616667,00:00:01.620518,0.371094
...,...,...,...,...
76,76,0.645455,00:00:01.637858,3.014430
47,47,0.645455,00:00:01.635687,3.021096
39,39,0.645455,00:00:01.619077,3.320122
40,40,0.645455,00:00:01.644892,3.005604


In [159]:
beta1 = solveRR(y_train, X_train, lam)

In [164]:
np.sum(np.sign(np.dot(X_train,beta1))==y_train)/len(y_train)

0.8738805970149254

In [165]:
np.sum(np.sign(np.dot(X_val,beta1))==y_val)/len(y_val)

0.6484848484848484

In [172]:
sol = []

for i in range(len(X_te)):
    result = np.sign(np.dot(X_te[i],beta1))
    if result == 1:
        sol.append([i,1])
    else:
        sol.append([i,0])
                     

In [176]:
d = pd.DataFrame(sol)
d.columns = ['Id','Bound']
d.head()

,Id,Bound
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0


In [177]:
d.to_csv('test_64_rr_solver.csv',index=False)